# Neo4j: Application of graph algorithms
COURSE: INFO9016-1 Advanced Databases

AUTHORS: 
- Emilien DE LA BRASSINE BONARDAUX 
- Yanis GEURTS
- Eri VAN DE VYVER


## Introduction

Graph databases have the particularity of representing data as nodes interconnected with edges, which respectively model entities and their relationships. These databases are based on the mathematical notion of graphs, resulting in a more flexible representation of data than relational databases. This representation allows to design algorithms more efficiently to answer different kinds of questions using graph properties and traversal methods from graph theory.[1]

In this tutorial, Neo4j has been chosen to illustrate the usage of graph databases, since it allows to model and query graph databases more easily. It uses the Cypher query language, which is optimised for pattern matching and graph exploration  and combines it with already implemented graph algorithms such as PageRank and shortest path. Unlike relational databases that rely on tables and JOIN operations, Neo4j uses a native graph structure that directly stores relationships, enabling faster and more scalable queries across networks.[2]

Neo4j is used for a wide range of real-world applications such as in fraud detection, where it identifies suspicious patterns by analysing transaction graphs or in recommendation systems, where it leverages similarity and collaborative filtering to suggest products or content.

This tutorial uses a dataset of airport routes [3] to show how graph databases handle tasks that are more difficult for traditional databases, especially when analyzing connectivity, hubs, and optimal paths across complex networks.

## Setup

For following this tutorial, docker must already be installed. Once installed execute these lines : 

`docker pull neo4j`

`docker compose up`

It will create a Neo4j container running the database service.

Install the Neo4j python driver and import next libraries.

In [5]:
!pip install neo4j
from neo4j import GraphDatabase
import pandas as pd

Now that everything is set up, the actual python environment just need to connect to the Neo4j docker container.

In [8]:
# Define Connection Parameters
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "neo4j")  # Default credentials for Neo4j when no auth is set

# Establish Connection and Query Executor
try:
    driver = GraphDatabase.driver(URI, auth=AUTH)
    driver.verify_connectivity()
    print("Connection to Neo4j successful!")
except Exception as e:
    print(f"Connection failed. Error: {e}")

def run_cypher_query(query):
    with driver.session() as session:
        result = session.execute_write(lambda tx: tx.run(query).data())
        return result
def run_cypher_query_df(query, params={}):
    """
    Executes a Cypher query using the global 'driver'
    and returns the result as a pandas DataFrame.
    """
    try:
        df = driver.execute_query(
            query,
            parameters_=params,
            database_="neo4j", 
            result_transformer_=pd.DataFrame
        )
        return df
    
    except Exception as e:
        print(f"Query failed: {e}")
        return pd.DataFrame()

Connection to Neo4j successful!


Let’s begin by creating our graph database with the essential structure and data. We first create constraints and indexes to ensure data consistency (for example, each airport must have a unique IATA code). Then, we load nodes and their geo‑hierarchy from the airport-node-list.csv file, linking airports to their cities, regions, countries, and continents. Finally, we load routes from the iroutes-edges.csv file, creating relationships between airports and storing the distance of each route.

In [9]:
print("\nStep 1: Creating Constraints and Indexes")

CONSTRAINTS_AND_INDEXES = [
    "CREATE CONSTRAINT airports IF NOT EXISTS FOR (a:Airport) REQUIRE a.iata IS UNIQUE",
    "CREATE CONSTRAINT cities IF NOT EXISTS FOR (c:City) REQUIRE c.name IS UNIQUE",
    "CREATE CONSTRAINT regions IF NOT EXISTS FOR (r:Region) REQUIRE r.name IS UNIQUE",
    "CREATE CONSTRAINT countries IF NOT EXISTS FOR (c:Country) REQUIRE c.code IS UNIQUE",
    "CREATE CONSTRAINT continents IF NOT EXISTS FOR (c:Continent) REQUIRE c.code IS UNIQUE",
    "CREATE INDEX locations IF NOT EXISTS FOR (air:Airport) ON (air.location)"
]

try:
    for query in CONSTRAINTS_AND_INDEXES:
        run_cypher_query(query)
    print("Constraints and indexes created successfully.")

except Exception as e:
    print(f"Failed to create constraints/indexes: {e}")


print("\nStep 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv")

NODE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///airport-node-list.csv' AS row

MERGE (a:Airport {iata: row.iata})
MERGE (ci:City {name: row.city})
MERGE (r:Region {name: row.region})
MERGE (co:Country {code: row.country})
MERGE (con:Continent {name: row.continent})

MERGE (a)-[:IN_CITY]->(ci)
MERGE (a)-[:IN_COUNTRY]->(co)
MERGE (ci)-[:IN_COUNTRY]->(co)
MERGE (r)-[:IN_COUNTRY]->(co)
MERGE (a)-[:IN_REGION]->(r)
MERGE (ci)-[:IN_REGION]->(r)
MERGE (a)-[:ON_CONTINENT]->(con)
MERGE (ci)-[:ON_CONTINENT]->(con)
MERGE (co)-[:ON_CONTINENT]->(con)
MERGE (r)-[:ON_CONTINENT]->(con)

SET a.id = row.id,
    a.icao = row.icao,
    a.descr = row.descr,
    a.runways = toInteger(row.runways),
    a.longest = toInteger(row.longest),
    a.altitude = toInteger(row.altitude),
    a.latitude = toFloat(row.lat),
    a.longitude = toFloat(row.lon)
    // Removed any potential a.city property to keep city as a separate City label

RETURN count(a) AS airports_loaded
"""

results_nodes = run_cypher_query(NODE_LOADING_QUERY)
print(f"Airports and Geographies loaded. Total Airports: {results_nodes[0]['airports_loaded']}")
print("\nStep 3: Loading Routes from iroutes-edges.csv")

ROUTE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///iroutes-edges.csv' AS row

MATCH (source:Airport {iata: row.src})
MATCH (target:Airport {iata: row.dest})

MERGE (source)-[r:HAS_ROUTE]->(target)
ON CREATE SET r.distance = toFloat(row.dist)

RETURN count(r) AS routes_loaded
"""

results_routes = run_cypher_query(ROUTE_LOADING_QUERY)
print(f"Routes loaded. Total HAS_ROUTE relationships: {results_routes[0]['routes_loaded']}")


Step 1: Creating Constraints and Indexes
Constraints and indexes created successfully.

Step 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv
Airports and Geographies loaded. Total Airports: 3503

Step 3: Loading Routes from iroutes-edges.csv
Routes loaded. Total HAS_ROUTE relationships: 46389


## Advantages and limitations

Neo4j’s biggest strength compared to relational databases is how it handles connected queries. In a relational database, exploring relationships requiring multiple entities (like “find all airports reachable within three flights from Brussels”) requires multiple JOINs across tables, and the cost of those JOINs grows rapidly as the dataset become bigger. Neo4j, however, uses a native graph storage where relationships are stored. This means that algorithms as shortest paths are executed in constant time relative to the number of hops, rather than degrading with dataset size. As a result, queries that would take seconds or minutes in SQL can often be executed much faster in Neo4j. This will be shown in the last question of this notebook where we are showing how to execute the A* algorithm to find the shortest flight path. 

Neo4j also have built-in functions like PageRank or community detection that are optimised to run directly inside the database engine, avoiding the overhead of exporting data into external analytics tools. This makes Neo4j particularly advantageous for real-time applications such as fraud detection, recommendation engines and route optimisation, where speed is important. The main drawback is that for flat, tabular or purely transactional data, relational databases may still perform better.[4]



- MAYBE TALK MORE ABOUT THE LIMITATIONS ? 

## Features of Neo4j

TODO

## Real-World Use Case 

TO DO


## Running Graph Algorithms


### Presentation of the dataset

The airport routes dataset showcases Neo4j’s real-world use case in transportation and logistics. It contains 3503 airports and 46389 flight routes. It enables getting information like shortest paths between cities and route optimisation. This dataset is well suited for exploring network connectivity, route planning and airline performance analysis. The structure of the relation of the dataset can be seen in the figure below [3]: 
![](Images/Structure.png "Structure of the dataset")


### Data Exploration

In [10]:
# Explore basic statistics
with driver.session() as session:
    num_nodes = session.run("MATCH (n) RETURN count(n) AS c").single()["c"]
    num_rels = session.run("MATCH ()-[r]->() RETURN count(r) AS c").single()["c"]

print(f"Nombre de nœuds : {num_nodes}")
print(f"Nombre de relations : {num_rels}")

with driver.session() as session:
    labels = session.run("CALL db.labels()")
    print("Labels disponibles :")
    for record in labels:
        print("-", record["label"])


Nombre de nœuds : 8627
Nombre de relations : 73954
Labels disponibles :
- Airport
- City
- Region
- Country
- Continent


In [11]:
# Explorate some relationships
with driver.session() as session:
    rels = session.run("CALL db.relationshipTypes()")
    print("Types de relations :")
    for record in rels:
        print("-", record["relationshipType"])

Types de relations :
- IN_CITY
- IN_COUNTRY
- IN_REGION
- ON_CONTINENT
- HAS_ROUTE


In [12]:
# Explorate some nodes
label = "Airport" 
with driver.session() as session:
    result = session.run(f"MATCH (n:{label}) RETURN n LIMIT 10")
    df = pd.DataFrame([dict(record["n"]) for record in result])  
df

,altitude,descr,longest,iata,latitude,icao,id,runways,longitude
0,1026,Hartsfield - Jackson Atlanta International Air...,12390,ATL,33.636700,KATL,1,5,-84.428101
1,151,Anchorage Ted Stevens,12400,ANC,61.174400,PANC,2,3,-149.996002
2,542,Austin Bergstrom International Airport,12250,AUS,30.194500,KAUS,3,2,-97.669899
3,599,Nashville International Airport,11030,BNA,36.124500,KBNA,4,4,-86.678200
4,19,Boston Logan,10083,BOS,42.364300,KBOS,5,6,-71.005203
5,143,Baltimore/Washington International Airport,10502,BWI,39.175400,KBWI,6,3,-76.668297
6,14,Ronald Reagan Washington National Airport,7169,DCA,38.852100,KDCA,7,3,-77.037697
7,607,Dallas/Fort Worth International Airport,13401,DFW,32.896801,KDFW,8,7,-97.038002
8,64,Fort Lauderdale/Hollywood International Airport,9000,FLL,26.072599,KFLL,9,2,-80.152702
9,313,Washington Dulles International Airport,11500,IAD,38.944500,KIAD,10,4,-77.455803


In [13]:
# Explorate some relationships
with driver.session() as session:
    result = session.run("""
        MATCH (a:Airport)-[r:IN_CITY]->(b:City)
        RETURN a.iata AS source, b.name AS target, type(r) AS relation
        LIMIT 10
    """)
    df_rels = pd.DataFrame([record.data() for record in result])

df_rels 

,source,target,relation
0,ATL,Atlanta,IN_CITY
1,ANC,Anchorage,IN_CITY
2,AUS,Austin,IN_CITY
3,BNA,Nashville,IN_CITY
4,BOS,Boston,IN_CITY
5,BWI,Baltimore,IN_CITY
6,IAD,Washington D.C.,IN_CITY
7,DCA,Washington D.C.,IN_CITY
8,DAL,Dallas,IN_CITY
9,DFW,Dallas,IN_CITY


### Basic Cypher Query
To be able to use complex graph algorithm, one should be able to perform basic cypher query. This will be usefull to perform graph projections after or to retrieve information.

#### Find a specific node:
For this example, we are going to check for the information stored in the node that has "CRL" as iata. This node should contains information of Charleroi's airport. 

The *MATCH* clause is used to get the airport for which the iata is "CRL"

In [14]:
iata = "CRL"
with driver.session() as session:
    result = run_cypher_query_df(f"""MATCH (a:Airport {{iata: '{iata}'}}) RETURN a.descr, a.runways, a.altitude, a.longest, a.latitude, a.longitude""")
result

,0,1,2,3,4,5
0,Brussels South Charleroi Airport,1,614,8366,50.459202,4.45382


#### Find connected data
Here, we are going to check for all airports that can be joined from Brussels South Charleroi Airport. 

In Cypher, a relationship r from a to b is represented by (a)-[r]->(b). 


In [15]:
iata = "CRL"
with driver.session() as session:
    result = run_cypher_query_df(f"""MATCH (a1:Airport {{iata: '{iata}'}})-[:HAS_ROUTE]->(a2:Airport)
                                 RETURN a2.iata, a2.descr""")
result

,0,1
0,AGA,Al Massira Airport
1,RMU,Region of Murcia International Airport
2,SVQ,Sevilla Airport
3,LPA,Gran Canaria Airport
4,BIA,Bastia-Poretta Airport
...,...,...
112,CIA,Ciampino–G. B. Pastine International Airport
113,ESB,Esenboğa International Airport
114,ESU,Mogador Airport
115,KIV,Chişinău International Airport


#### Filter on relationship
We are now going to check, in all airports that has a direct route from Brussels Charleroi, which one are at more than 2000 km. 

A filter in cypher is done with the *WHERE* clause.

In [16]:
iata = "CRL"
with driver.session() as session:
    result = run_cypher_query_df(f"""MATCH (a1:Airport {{iata: '{iata}'}})-[r:HAS_ROUTE]->(a2:Airport)
                                 WHERE r.distance > 2000
                                 RETURN a2.iata, a2.descr""")
result

,0,1
0,TLV,"Tel Aviv, Ben Gurion International Airport"
1,FDF,Martinique Aimé Césaire International Airport
2,VDA,Ovda International Airport
3,HKG,Hong Kong - Chek Lap Kok International Airport
4,SSH,Sharm El Sheikh International Airport
5,HRG,Hurghada International Airport


### Using the GDS library

Typically, when one want to use graph algorithms, the Graph Data Science library is the best option. Indeed, it provides efficiently implemented versions of graph algorithms. In addition, it offers an intuitive framework that connects with Neo4j's graph database[5]. 


In cypher, the way to call a graph algorithm with GDS is as follow : 

`CALL gds[.<tier>].<algorithm>.<execution-mode>[.<estimate>](`

`graphName: String,`

  `configuration: Map)`

Where graphName is a projected graph keeping the attribute we need for the our algorithms

### Path Finding Algorithms

#### The Dijkstra Algorithm

For this first example, we are going to apply the Dijkstra algorithm to find the shortest path between two country. To compute this, we are going to need a graph in which we will only keep the city and the relations between their respective airports. This last relation is derived from the relation linking airports in those city. 





For this first algorithm, we are going to apply the PageRank algorithm on a graph containing the different country with their airport. This will allow us to get the most important country, which is determined by considering links with other country that are considered more important.

To do that, we start by creating a projected graph with the different country as note and the relationship between two country as relationship. This last relation is derived from the relation linking two country. These kind of request are one the main advantages of Cypher.

The second step is to apply the PageRank algorithm on this graph. Once it is done, we are taking the nodeId and score of each node. With this, we can order the country by score descending (resp. ascending) with the keyword DESC (resp. ASC).

In [17]:
# Let's now create a projected graph with only the needed nodes and relationships
with driver.session() as session:
    run_cypher_query_df("CALL gds.graph.drop('city_routes') YIELD graphName")
    result = run_cypher_query_df("""
        CALL gds.graph.project.cypher(
        'city_routes', 
        'MATCH (c:City) RETURN id(c) AS id',
        'MATCH (c1:City)<-[:IN_CITY]-(a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)-[:IN_CITY]->(c2:City) RETURN id(c1) AS source, id(c2) AS target, r.distance AS dist'
        )
        YIELD graphName, nodeCount, relationshipCount
    """)
    print(result)
    

Query failed: {neo4j_code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.drop`: Caused by: org.neo4j.gds.core.loading.GraphNotFoundException: Graph with name `city_routes` does not exist on database `neo4j`. It might exist on another database.} {gql_status: 52N37} {gql_status_description: error: procedure exception - procedure execution error. Execution of the procedure gds.graph.drop() failed.}


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. gds.graph.project.cypher is deprecated. It is replaced by gds.graph.project Cypher projection as an aggregation function.', position=<SummaryInputPosition line=2, column=9, offset=9>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 9, 'line': 2, 'column': 9}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n        CALL gds.graph.project.cypher(\n        'city_routes', \n        'MATCH (c:City) RETURN id(c) AS id',\n        'MATCH (c1:City)<-[:IN_CITY]-(a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)-[:IN_CITY]->(c2:City) RETURN id(c1) AS source, id(c2) AS target, r.distance AS dist'\n        )\n 

             0     1      2
0  city_routes  3359  46389


For this example, we are going to join two city from islander country that are located on the opposite side of the world so that the shortest path still has a few hops. Here we chose Funafuti in Tuvalu and Basseterre in Saint Kitts. 

In [18]:
with driver.session() as session:
    result = run_cypher_query_df("""
    MATCH (source:City {name: 'Funafuti'}), (target:City {name: 'Basseterre'})
    CALL gds.shortestPath.dijkstra.stream('city_routes', {
        sourceNode: source,
        targetNodes: target,
        relationshipWeightProperty: 'dist'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS sourceNodeName,
        gds.util.asNode(targetNode).name AS targetNodeName,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
        costs
    ORDER BY index
    """)
    for _, row in result.iterrows():
        for i in range(len(row)):
            print(f"{i}: {row[i]}")

0: Funafuti
1: Basseterre
2: 3796.0
3: ['Funafuti', 'Nausori', 'Auckland', 'Hamilton', 'Montreal', 'Toronto', 'Kingston', 'Santo Domingo', 'San Juan', 'Basseterre']
4: [0.0, 659.0, 1988.0, 2054.0, 2398.0, 2704.0, 2859.0, 3327.0, 3567.0, 3796.0]


#### The A* Algorithm
The A* algorithm is also an algorithm that is made to compute the shortest path between two nodes in a graph with weighted relationships. The difference with dijkstra is that it uses an heuristic. In our case, the heuristic will use the latitude and longitude of the airport. 

Let's first create a projected graph that contains the airports and their connections. 



In [19]:
# Let's now create a projected graph with only the needed nodes and relationships
with driver.session() as session:
    #run_cypher_query_df("CALL gds.graph.drop('airport_relations') YIELD graphName")
    result = run_cypher_query_df("""
    CALL gds.graph.project.cypher(
        'airport_relations',
        'MATCH (a:Airport)
        RETURN id(a) AS id,
                a.latitude AS latitude, 
                a.longitude AS longitude',
        'MATCH (a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)
        RETURN id(a1) AS source, id(a2) AS target, 
                r.distance AS distance'
    )
    YIELD graphName, nodeCount, relationshipCount
    """)
    print(result)

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. gds.graph.project.cypher is deprecated. It is replaced by gds.graph.project Cypher projection as an aggregation function.', position=<SummaryInputPosition line=2, column=5, offset=5>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 5, 'line': 2, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    CALL gds.graph.project.cypher(\n        'airport_relations',\n        'MATCH (a:Airport)\n        RETURN id(a) AS id,\n                a.latitude AS latitude, \n                a.longitude AS longitude',\n        'MATCH (a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)\n        RETURN id(a1)

                   0     1      2
0  airport_relations  3503  46389


The next step is to use the Algorithm with our graph. For the purpose of the example, we will try to use two city that are poorly connected. We are able to find them by using the PageRank algorithm. We are going to try to find the shortest path between Pago Pago and Mount Pleasant. Both are located in small island, the first one in Oceania and the second near Argentina. 

In [20]:
with driver.session() as session:
    result = run_cypher_query_df("""
    MATCH (source:Airport {iata : 'PPG'}), (target:Airport {iata: 'MPN'})
    CALL gds.shortestPath.astar.stream('airport_relations', {
        sourceNode: id(source),
        targetNode: id(target),
        latitudeProperty: 'latitude',
        longitudeProperty: 'longitude',
        relationshipWeightProperty: 'distance'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).iata AS sourceNodeName,
        gds.util.asNode(targetNode).iata AS targetNodeName,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).iata] AS nodeNames,
        costs
    ORDER BY index
    """)
    for _, row in result.iterrows():
        for i in range(len(row)):
            print(f"{i}: {row[i]}")

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=4, column=21, offset=157>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 157, 'line': 4, 'column': 21}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    MATCH (source:Airport {iata : 'PPG'}), (target:Airport {iata: 'MPN'})\n    CALL gds.shortestPath.astar.stream('airport_relations', {\n        sourceNode: id(source),\n        targetNode: id(target),\n        latitudeProperty: 'latitude',\n        longitudeProperty: 'longitude',\n        relationshipWeight

0: PPG
1: MPN
2: 12202.0
3: ['PPG', 'HNL', 'PPT', 'IPC', 'SCL', 'PUQ', 'MPN']
4: [0.0, 2610.0, 5352.0, 7990.0, 10320.0, 11674.0, 12202.0]


The output shows us that the algorithm found the shortest path between both airports. 

### Centrality Algorithm

#### Betweenness Centrality
The betweenness centrality is a measure of the amount of influence a node has over the flow of information in a graph. We are going to use it to see which airports are the most critical bridges. 

In [21]:
with driver.session() as session:
    result = run_cypher_query_df("""
    CALL gds.betweenness.stream('airport_relations')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).iata AS iata, gds.util.asNode(nodeId).descr AS descr, score
    ORDER BY score DESC
    LIMIT 10
    """)
    print(result)

     0                                        1              2
0  DXB              Dubai International Airport  390958.585096
1  LAX        Los Angeles International Airport  368734.104102
2  CDG                  Paris Charles de Gaulle  365259.650707
3  PEK    Beijing Capital International Airport  340393.582884
4  IST           Istanbul International Airport  339441.886007
5  ORD     Chicago O'Hare International Airport  326830.237013
6  ANC                    Anchorage Ted Stevens  298891.743319
7  FRA                        Frankfurt am Main  285270.758162
8  DFW  Dallas/Fort Worth International Airport  277537.092487
9  AMS               Amsterdam Airport Schiphol  265922.715158


The result obtained was expected. The airports with the highest score are: Dubai, Los Angeles, Paris, Beijing ans Istanbul. Istanbul and Dubai are both located between Asia, Europe and Africa. It is often mandatory to fly through them to go from one continent to another. Concerning Paris, Beijing, and Los Angeles, they are the primary gateways for their respective continent which are the three most developed continent. The conclusion that can be directly infered from this result is that: If, for some reason, one of these airport were to close, the impact on the global connectivity would be severe. Travelers would need to take other longer, less efficient routes.

Airport with a score of 0 represents small regional airports. This can be verified by changing "DESC" by "ASC" in the request. 

In [22]:
driver.close()
print("\nNeo4j connection closed.")


Neo4j connection closed.


## Conclusion

TO DO

## Work distribution

The work was distributed as follows:
- Emilien DE LA BRASSINE BONARDAUX 
- Yanis GEURTS: General structure, selection of the database, setup of the tutorial, eda, simple query, Dijkstra, A*, Betweenness Centrality
- Eri VAN DE VYVER: General tutorial structure, Intoduction, Advantages and limitations

## Declaration on Generative AI

During the preparation of this work, the authors used chatGPT-5 and Mistral Large to: Grammar and spelling check, improve writing style, and peer review simulation. After using these tools/services, the authors reviewed and edited the content as needed and take full responsibility for the publication’s content.

## Bibliography 

1. Debruyne Christophe. "INFO9016-1 Advanced Databases - Chapter 4: NoSQL Part 2". University of Liège, 2025.
2. Baylor Corydon and Enzo Htet. "What Are the Different Types of Graph Algorithms & When to Use Them?", 29 May 2025, url: https://neo4j.com/blog/graph-data-science/graph-algorithms/. Accessed 14 Nov 2025.
3. Neo4j. Airport Routes Graph Example. GitHub, url: https://github.com/neo4j-graph-examples/airport-routes. Accessed 14 Nov 2025.
4. Dadashzadeh, Ali. "Neo4j vs. SQL: Unlocking the Power of Graph-Based Data Modeling." DEV Community, 19 Feb 2025, url: https://dev.to/ali_dz/neo4j-vs-sql-unlocking-the-power-of-graph-based-data-modeling-33da. Accessed 14 Nov 2025.
5. Kolla, S. (2020). Neo4j Graph Data Science (GDS) library: Advanced analytics on connected data. International Journal of Advanced Research in Engineering and Technology, 11(8), 1077-1086.